In [37]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Network Science Analytics
Lab 1: Centrality Criteria 
21 December 2020
"""
from helper import *
import numpy as np
import scipy.sparse as sparse
from scipy.stats.stats import pearsonr
import matplotlib.pyplot as plt
import networkx as nx

In [34]:
import tweepy
import csv
import pandas as pd
import time
import sys
import itertools
from datetime import datetime
import sys
import networkx as nx
import plotly as py
import plotly.graph_objects as go
from plotly.graph_objs import *
import matplotlib.pyplot as plt
import scipy.sparse as sparse
from scipy.stats.stats import pearsonr
%matplotlib inline

In [2]:
#Twitter API credentials
#consumer_key = "BmvoUUcOUXhPxRR8uRC2TgKoW"
#consumer_secret = "bNV6inRgeUSSVerytnnnTPveW8iM9GM0dwryZyiUKmYy436D1I"
#access_key = "2969993776-b9Ui7fVJjW7gYId2C0kSGo5mN4ki93HSGEn6jx0"
#access_secret = "N5ER33zjeIqfl5918MWTHLWbZzuBGfGL0FeSfNGvSsrvZ"

In [3]:
#OAUTH_KEYS = {'consumer_key':consumer_key, 'consumer_secret':consumer_secret, 'access_token_key':access_key, 'access_token_secret':access_secret}
#auth = tweepy.OAuthHandler(OAUTH_KEYS['consumer_key'], OAUTH_KEYS['consumer_secret'])
#api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [4]:
def get_names_for_download(dates):
    pwd = '/home/b00762654/twitter_None'
    for i in range(len(dates) - 1):
        csv_name = pwd + '/data_'+ str(dates[i]) + '_' + str(dates[i+1]) + '.csv'
        print(csv_name)

## Functions

In [294]:
def load_folder_as_dict(dates, prefix='data/'):
    dict_df = dict()
    for i in range(len(dates) - 1):
        csv_name = prefix + 'data_'+ str(dates[i]) + '_' + str(dates[i+1]) + '.csv'
        df_name = 'df_' + str(dates[i])
        df = pd.read_csv(csv_name)
        df = df[df.is_retweeted == True]
        dict_df[df_name] = df
    return dict_df

In [295]:
def get_stats_on_df(dict_df):
    l_1 = []
    l_2 = []

    for key in dict_df.keys():
        df = dict_df[key]
        l_user_rt = list(df[df.is_retweeted == True].user_retweeted.unique())
        l_users = list(df[df.is_retweeted == True].username.unique())
        l_tot = l_users.copy()
        print('Nb unique user retweeted for ', key, ' : ', len(l_user_rt))
        print('Nb unique users who tweet for ', key, ' : ', len(l_users))
        l_tot.extend(l_user_rt)
        print('Nb unique users for ', key, ' : ', len(list(set(l_tot))))
        print()
        l_1.extend(l_user_rt)
        l_2.extend(l_users)
    print()
    print('Nb of unique users retweeted for al these df :', len(list(set(l_1))))
    print('Nb of unique users who tweet for al these df :', len(list(set(l_2))))
    return

In [296]:
def define_graph_from_df(df):
    G = nx.from_pandas_edgelist(
        df,
        source = 'username', 
        target = 'user_retweeted',
        create_using = nx.DiGraph())
    return G

In [304]:
def define_graphs(dict_df, stats=False):
    graph_dict = dict()
    for key in dict_df.keys():
        graph_dict[key] = define_graph_from_df(dict_df[key])
        if stats:
            print('Nodes in RT network:', len(graph_dict[key].nodes()))
            print('Edges in RT network:', len(graph_dict[key].edges()), '\n')
    return graph_dict

#### Define dates for which we have data

In [305]:
dates = ['2020-03-28', '2020-03-29', '2020-03-30', '2020-03-31',
        '2020-04-01', '2020-04-02', '2020-04-03', '2020-04-04', '2020-04-05', '2020-04-06']

#### Define dictionaries of dataframes and graphs

In [306]:
df_dict = load_folder_as_dict(dates, prefix='data/')

In [309]:
graph_dict = define_graphs(df_dict, stats=False)

#### Call a dataframe and graph according to a key

In [310]:
key = list(df_dict.keys())[0]
print(key)

df_2020-03-28


In [311]:
df = df_dict[key]
G = graph_dict[key]

## Nodes Information

In [321]:
def compute_degree_centrality(G):
    degree_centrality = {}
    for node in G.nodes():
        degree_centrality[node] = [G.degree(node),  G.in_degree(node), G.out_degree(node)]
    return degree_centrality

In [347]:
def define_df_nodes(df, G):
    
    df_nodes = df[df.is_retweeted == True]
    df_nodes.loc[:, 'count_rt_day'] = df_nodes.groupby('user_retweeted').user_retweeted.transform('count')
    df_nodes.loc[:, 'count_t_day'] = df_nodes.groupby('username').username.transform('count')

    df_nodes_1 = df_nodes[['user_retweeted', 'count_rt_day']].drop_duplicates()
    df_nodes_1.columns = ['username', 'count_rt_day']
    df_nodes_2 = df_nodes[['username', 'count_t_day']].drop_duplicates()
    
    df = pd.merge(df_nodes_1, df_nodes_2, on='username', how='outer')
    #compute closeness centrality for each nodes
    df_closeness = pd.DataFrame(nx.closeness_centrality(G).items(), columns=['username', 'closeness_centrality'])
    #compute harmonic centrality for each nodes
    df_harmonic = pd.DataFrame(nx.harmonic_centrality(G).items(), columns=['username', 'harmonic_centrality'])
    #comput the degree centrality for each nodes
    df_degree = pd.DataFrame(compute_degree_centrality(G)).T.reset_index()
    df_degree.columns = ['username', 'degree', 'in_degree', 'out_degree']
    
    df = pd.merge(df, df_closeness, on='username', how='outer')
    df = pd.merge(df, df_harmonic, on='username', how='outer')
    df = pd.merge(df, df_degree, on='username', how='outer')
    df.fillna(0, inplace=True)
    return df

In [348]:
df_test = define_df_nodes(df, G)

In [349]:
df_test.sort_values('count_rt_day', ascending=False)

,username,count_rt_day,count_t_day,closeness_centrality,harmonic_centrality,degree,in_degree,out_degree
1,PoliceSynergie,2228.0,0.0,0.268952,2228.5,2228,2228,0
14,acupoftim,601.0,0.0,0.072072,599.5,597,597,0
19,AllanBARTE,585.0,0.0,0.068324,566.0,566,566,0
5,EstherBenbassa,483.0,0.0,0.054925,455.0,455,455,0
34,sararobe93,377.0,1.0,0.045389,376.0,378,377,1
...,...,...,...,...,...,...,...,...
3154,YuffieRules,0.0,1.0,0.000000,0.0,1,0,1
3153,cesarcardo,0.0,1.0,0.000000,0.0,1,0,1
3152,jesuispresque,0.0,1.0,0.000000,0.0,1,0,1
3151,kedipity,0.0,1.0,0.000000,0.0,1,0,1


## Edge Information

In [350]:
df

,timestamp,username,count_rt,text,is_retweeted,user_retweeted
0,2020-03-28 23:59:40,Galvzerrr,72,RT @NoamCartozo: J'ai encore rêvé d'elle - Il ...,True,NoamCartozo
1,2020-03-28 23:58:12,SArrow9578,3907,RT @PoliceSynergie: Guet-apens cette nuit cont...,True,PoliceSynergie
3,2020-03-28 23:57:37,Rausteen_,72,RT @NoamCartozo: J'ai encore rêvé d'elle - Il ...,True,NoamCartozo
4,2020-03-28 23:57:27,cyrilboji,114,RT @ScpnCommissaire: Enfin ! les crèches et le...,True,ScpnCommissaire
5,2020-03-28 23:57:24,Desperadjo,55,RT @JeanLucLemoine: J11 Astuce bonheur : à ma ...,True,JeanLucLemoine
...,...,...,...,...,...,...
9733,2020-03-28 00:01:44,AgnesDecagny,628,RT @30millionsdamis: 😱😡#OnMarcheSurLaTête‼️A p...,True,30millionsdamis
9734,2020-03-28 00:01:41,ChicanosDeDios,146,RT @BlancheurO: Notre gouv. ferait bien de méd...,True,BlancheurO
9735,2020-03-28 00:01:29,njee26,3907,RT @PoliceSynergie: Guet-apens cette nuit cont...,True,PoliceSynergie
9736,2020-03-28 00:01:27,VeroniqueBlan20,74,RT @pierrejovanovic: voilà ce qui risque de se...,True,pierrejovanovic


In [ ]:
def define_df_nodes(df, G):
    
    df_edges = df[df.is_retweeted == True]
    df_edges.loc[:, 'count_rt_day'] = df_edges.groupby('user_retweeted').user_retweeted.transform('count')
    df_edges.loc[:, 'count_t_day'] = df_edges.groupby('username').username.transform('count')
    
    #compute closeness centrality for each nodes
    df_closeness = pd.DataFrame(nx.closeness_centrality(G).items(), columns=['username', 'closeness_centrality'])
    #compute harmonic centrality for each nodes
    df_harmonic = pd.DataFrame(nx.harmonic_centrality(G).items(), columns=['username', 'harmonic_centrality'])
    #comput the degree centrality for each nodes
    df_degree = pd.DataFrame(compute_degree_centrality(G)).T.reset_index()
    df_degree.columns = ['username', 'degree', 'in_degree', 'out_degree']
    
    df = pd.merge(df, df_closeness, on='username', how='outer')
    df = pd.merge(df, df_harmonic, on='username', how='outer')
    df = pd.merge(df, df_degree, on='username', how='outer')
    df.fillna(0, inplace=True)
    return df

## Clustering